In [1]:
!pip install s3fs # Needed so that pandas can read and right to s3. pandas uses s3fs for handling S3 connections 
!pip install unicodedata2 # needed to remove accented characters
!pip install spacy
!pip install wordcloud # to generate word cloud
!pip install gensim
!python3 -m spacy download en_core_web_sm
!python3 -m spacy download en
!pip install boto3

  Using cached botocore-1.17.44-py2.py3-none-any.whl (6.5 MB)
ERROR: boto3 1.16.33 has requirement botocore<1.20.0,>=1.19.33, but you'll have botocore 1.17.44 which is incompatible.
  Attempting uninstall: botocore
    Found existing installation: botocore 1.19.33
    Uninstalling botocore-1.19.33:
      Successfully uninstalled botocore-1.19.33


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/Users/daniellescaramella/opt/anaconda3/lib/python3.8/site-packages/en_core_web_sm
-->
/Users/daniellescaramella/opt/anaconda3/lib/python3.8/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


     |████████████████████████████████| 7.1 MB 2.1 MB/s eta 0:00:01
ERROR: aiobotocore 1.1.2 has requirement botocore<1.17.45,>=1.17.44, but you'll have botocore 1.19.35 which is incompatible.
  Attempting uninstall: botocore
    Found existing installation: botocore 1.17.44
    Uninstalling botocore-1.17.44:
      Successfully uninstalled botocore-1.17.44


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import csv
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import seaborn as sns
import re
import boto3
import string
from io import StringIO
import spacy
from spacy.lang import punctuation
import unicodedata2
from contractions import CONTRACTION_DICT
#from html.parser import HTMLParser
#from pathlib import Path
from wordcloud import WordCloud, STOPWORDS
import nltk
nltk.download('stopwords')
from nltk.tokenize import ToktokTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk import FreqDist
from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.util import ngrams
from collections import defaultdict
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
import gensim
from gensim.models import word2vec
from gensim.models.fasttext import FastText
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import STOPWORDS


import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
sns.set(style='darkgrid',palette='Dark2',rc={'figure.figsize':(9,6),'figure.dpi':100})

tokenizer = ToktokTokenizer()
punctuation = string.punctuation + '”' + '“' + '–' + '““' + "’’" + '”'
stopword = stopwords.words('english')
wordnet_lemmatizer = WordNetLemmatizer()
stop=set(stopwords.words('english'))
snowball_stemmer = SnowballStemmer('english')
stopwords = set(STOPWORDS)
nlp = spacy.load('en',parse=True,tag=True, entity=True)
# if you get error from the above , you will need to install the following to stage the proper file:
#python -m spacy download en_core_web_sm
#python -m spacy download en

/Users/daniellescaramella/opt/anaconda3/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/daniellescaramella/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
#Path to the huge csv file
input_data = 's3://sagemaker-studio-zvdmh7fos3/news_cleaned_2018_02_13.csv'

In [ ]:
%%time

df_list = [] # list to hold the batch dataframe
for chunk in tqdm (pd.read_csv(input_data, chunksize=chunksize, encoding='utf-8', engine='python')):
    # append the chunk to list 
    df_list.append(chunk)

In [ ]:
%%time
# Merge all the chunked dataframes into one dataframe
df = pd.concat(df_list)

# Delete the chunked dataframe list to release memory
del df_list

# See what we have loaded
df.info()

In [ ]:
df.head()

In [ ]:
# list of columns to drop from the datasets
cols_to_drop = ['id','Unnamed: 0', 'url', 'source', 'summary', 'keywords', 'meta_description','updated_at', 'inserted_at','tags','meta_keywords']

In [ ]:
# drop the columns and check the memory usage
df.drop(cols_to_drop, axis=1, inplace=True)
df.info()

In [ ]:
df.shape
print('The dataframe contains:', df.shape[0] ,'rows', 'and', df.shape[1], 'columns')

In [ ]:
# list of classes in the type column that need to be dropped
classes_to_drop = ['rumor','bias','political','conspiracy','unknown','clickbait','junksci','satire','unreliable','hate']

# Getting the indexes for the classes
index_names = df[ df['type'].isin(classes_to_drop)].index 

# dropping the rows based on the column indexes
df.drop(index_names, inplace = True) 

In [ ]:
#Labeling the class
class_labels = ['fake','real']

#replace the class
df['label'] = df['type'].replace(['fake','reliable'], class_labels)

# check the column values
df['label'].value_counts(dropna=False, ascending=False)

In [ ]:
import s3fs

s3 = s3fs.S3FileSystem(anon=False) # publicly-available,  # uses default credentials
#s3 = s3fs.S3FileSystem(anon=False) # credentials explicitly supplied
# Use 'w' for py3, 'wb' for py2
with s3.open('(cleaned_frl_fake2.csv','w') as f:
    df.to_csv(f) # Should have been df.to_csv(f, index=False)

In [ ]:
def write_dataframe_to_csv_on_s3(dataframe, file_key):
    """ Write a dataframe to a CSV on S3 """
    """Sends csv to S3 from memory"""
    print("Writing {} records to {}".format(len(dataframe), file_key))
     # Create buffer
    csv_buffer = StringIO()
    # Write dataframe to buffer
    dataframe.to_csv(csv_buffer, index=False)
    # Create S3 object
    s3_resource = boto3.resource("s3")
    # Write buffer to S3 object
    s3_resource.Object(myBucket, file_key).put(Body=csv_buffer.getvalue())

In [ ]:
write_dataframe_to_csv_on_s3(df, 'cleaned_frl_fake.csv')